# <center>Geometric Methods in Machine Learning<br/><br/>Poincaré Embeddings<br/><br/>Zakarya Ali et Samir Tanfous</center>

Nous étudions l'article [Poincaré Embeddings for Learning Hierarchical Representations](https://arxiv.org/pdf/1705.08039.pdf) de Maximilian Nickel et Douwe Kiela.

## Prérequis
On fait l'import du code et des données nécessaires

In [60]:
import os
import numpy as np
import csv
import random
import time
import pickle

In [61]:
wordnet_mammal_file = 'data/wordnet_mammal_hypernyms.tsv'

In [62]:
def load_wordnet(wordnet_path):
    network = {}
    with open(wordnet_path, 'r') as f:
        reader = csv.reader(f, delimiter='\t')
        for row in reader:
#             assert len(row) == 2, 'Hypernym pair has more than two items'
            if row[0] in network:
                network[row[0]].append(row[1])
            else:
                network.update({row[0]:[row[1]]})
    return network

In [63]:
network = load_wordnet(wordnet_mammal_file)
network

{'kangaroo.n.01': ['marsupial.n.01',
  'mammal.n.01',
  'metatherian.n.01',
  'kangaroo.n.01'],
 'domestic_goat.n.01': ['even-toed_ungulate.n.01',
  'ruminant.n.01',
  'domestic_goat.n.01',
  'goat.n.01',
  'placental.n.01',
  'bovid.n.01',
  'ungulate.n.01',
  'mammal.n.01'],
 'rock_squirrel.n.01': ['ground_squirrel.n.02',
  'rock_squirrel.n.01',
  'squirrel.n.01',
  'rodent.n.01',
  'mammal.n.01',
  'placental.n.01'],
 'vizsla.n.01': ['dog.n.01',
  'placental.n.01',
  'sporting_dog.n.01',
  'mammal.n.01',
  'carnivore.n.01',
  'canine.n.02',
  'pointer.n.04',
  'hunting_dog.n.01',
  'vizsla.n.01'],
 'dandie_dinmont.n.01': ['mammal.n.01',
  'dandie_dinmont.n.01',
  'terrier.n.01',
  'dog.n.01',
  'hunting_dog.n.01',
  'canine.n.02',
  'placental.n.01',
  'carnivore.n.01'],
 'broodmare.n.01': ['horse.n.01',
  'ungulate.n.01',
  'odd-toed_ungulate.n.01',
  'placental.n.01',
  'broodmare.n.01',
  'mare.n.01',
  'mammal.n.01',
  'equine.n.01'],
 'spotted_skunk.n.01': ['spotted_skunk.n.01'

In [64]:
def dist(vec1, vec2): 
    """Distance de Poincaré (équation 1 de l'article)"""
    return 1 + 2*np.dot(vec1 - vec2, vec1 - vec2)/ \
             ((1-np.dot(vec1, vec1))*(1-np.dot(vec2, vec2)) + STABILITY)

In [70]:
def partial_der(theta, x, gamma): 
    """Dérivée partielle par rapport à thete (equation 4 de l'article)"""
    alpha = (1.0-np.dot(theta, theta))
    norm_x = np.dot(x, x)
    beta = (1-norm_x)
    gamma = gamma
    return 4.0/(beta * np.sqrt(gamma*gamma - 1) + STABILITY)*((norm_x- 2*np.dot(theta, x)+1)/(pow(alpha,2)+STABILITY)*theta - x/(alpha + STABILITY))

In [71]:
def update(emb, error_):
    """Equation d'update (equation 5)"""
    try:
        update =  lr*pow((1 - np.dot(emb,emb)), 2)*error_/4
        emb = emb - update
        if (np.dot(emb, emb) >= 1):
            emb = emb/sqrt(np.dot(emb, emb)) - STABILITY
        return emb
    except Exception as e:
        print (e)

# 1. Entrainement

In [72]:
#Initialisation des variables

# embedding of nodes of network
emb = {}
embedding_size = 5 #Dimensionality of the trained vectors

#epochs
num_epochs = 50

#num_negs
num_negs = 20

#learning rate
lr = 0.01

STABILITY = 0.00001 # to avoid overflow while dividing

In [73]:
# Randomly uniform distribution
for a in network:
    for b in network[a]:
        emb[b] = np.random.uniform(low=-0.001, high=0.001, size=(embedding_size,))
    emb[a] = np.random.uniform(low=-0.001, high=0.001, size=(embedding_size,))

vocab = list(emb.keys())
random.shuffle(vocab)

In [ ]:
# the leave nodes are not connected to anything
for a in emb:
    if not a in network:
        network[a] = []
        
# The plot of initialized embeddings
# plotall("init")

last_time = time.time()
for epoch in range(num_epochs):
    # pos2 is related to pos1
    # negs are not related to pos1
    for pos1 in vocab:
        if not network[pos1]: # a leaf node
            continue
        pos2 = random.choice(network[pos1]) # pos2 and pos1 are related
        dist_p_init = dist(emb[pos1], emb[pos2]) # distance between the related nodes
        if (dist_p_init > 700): # this causes overflow, so I clipped it here
            print ("got one very high") # if you have reached this zone, the training is unstable now
            dist_p_init = 700
        elif (dist_p_init < -700):
            print ("got one very high")
            dist_p_init = -700
        dist_p = np.cosh(dist_p_init) # this is the actual distance, it is always positive
        # print ("distance between related nodes", dist_p)
        negs = [] # pairs of not related nodes, the first node in the pair is `pos1`
        dist_negs_init = [] # distances without taking cosh on it (for not related nodes)
        dist_negs = [] # distances with taking cosh on it (for not related nodes)
        while (len(negs) < num_negs):
            neg1 = pos1
            neg2 = random.choice(vocab)
            if not (neg2 in network[neg1] or neg1 in network[neg2] or neg2 == neg1): # neg2 should not be related to neg1 and vice versa
                dist_neg_init = dist(emb[neg1], emb[neg2])
                if (dist_neg_init > 700 or dist_neg_init < -700): # already dist is good, leave it
                    continue
                negs.append([neg1, neg2])
                dist_neg = np.cosh(dist_neg_init)
                dist_negs_init.append(dist_neg_init) # saving it for faster computation
                dist_negs.append(dist_neg)
                # print ("distance between non related nodes", dist_neg)
        loss_den = 0.0
        # eqn6
        for dist_neg in dist_negs:
            loss_den += np.exp(-1*dist_neg)
        loss = -1*dist_p - np.log(loss_den + STABILITY)
        # derivative of loss wrt positive relation [d(u, v)]
        der_p = -1
        der_negs = []
        # derivative of loss wrt negative relation [d(u, v')]
        for dist_neg in dist_negs:
            der_negs.append(np.exp(-1*dist_neg)/(loss_den + STABILITY))
        # derivative of loss wrt pos1
        der_p_pos1 = der_p * partial_der(emb[pos1], emb[pos2], dist_p_init)
        # derivative of loss wrt pos2
        der_p_pos2 = der_p * partial_der(emb[pos2], emb[pos1], dist_p_init)
        der_negs_final = []
        for (der_neg, neg, dist_neg_init) in zip(der_negs, negs, dist_negs_init):
            # derivative of loss wrt second element of the pair in neg
            der_neg1 = der_neg * partial_der(emb[neg[1]], emb[neg[0]], dist_neg_init)
            # derivative of loss wrt first element of the pair in neg
            der_neg0 = der_neg * partial_der(emb[neg[0]], emb[neg[1]], dist_neg_init)
            der_negs_final.append([der_neg0, der_neg1])
        # update embeddings now
        emb[pos1] = update(emb[pos1], -1*der_p_pos1)
        emb[pos2] = update(emb[pos2], -1*der_p_pos2)
        for (neg, der_neg) in list(zip(negs, der_negs_final)):
            emb[neg[0]] = update(emb[neg[0]], -1*der_neg[0])
            emb[neg[1]] = update(emb[neg[1]], -1*der_neg[1])
    print('Epoch #%d, time taken: %.2f seconds' % (epoch + 1, time.time() - last_time))
    last_time = time.time()
pickle.dump(emb, open('data/train.pickle', 'wb'))

# 2. Résultats